<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [30]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=Warning);

In [31]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


### ¿Cuántas observaciones hay?

In [32]:
print(f"Tenemos {data.shape[0]:,d} observaciones.")

Tenemos 1,000 observaciones.


### ¿Cuales y cuántas categorías únicas de productos hay?

In [33]:
data["product_category"].unique()

array(['jewelry', 'video_games'], dtype=object)

###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [34]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

In [35]:
data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [36]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

In [57]:
#Train the topic model
intial_num_topics = 2
max_num_topics = 5
_passes = 100
models = []
index = 1

for _num_topics in range(intial_num_topics, max_num_topics + 1):
    print(f'Entrenando modelo {index} para {_num_topics} temas con {_passes} pasos...')
    _model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=_num_topics, passes=_passes)
    models.append(_model)
    index += 1

Entrenando modelo 1 para 2 temas con 100 pasos...
Entrenando modelo 2 para 3 temas con 100 pasos...
Entrenando modelo 3 para 4 temas con 100 pasos...
Entrenando modelo 4 para 5 temas con 100 pasos...


### Imprimir topics de cada modelo

In [58]:
topics_to_print = 5
_index = 1
for model in models:
    print(f"Temas para modelo {_index}")
    print("*"*90)
    print(model.print_topics(topics_to_print))
    print()
    _index += 1

Temas para modelo 1
******************************************************************************************
[(0, '0.031*"bien" + 0.024*"calidad" + 0.019*"bonito" + 0.015*"llegado" + 0.014*"foto" + 0.014*"día" + 0.011*"bonitos" + 0.011*"producto" + 0.011*"pulsera" + 0.011*"regalo"'), (1, '0.028*"juego" + 0.023*"precio" + 0.021*"si" + 0.019*"bien" + 0.016*"producto" + 0.015*"buena" + 0.015*"calidad" + 0.014*"buen" + 0.012*"puede" + 0.011*"perfecto"')]

Temas para modelo 2
******************************************************************************************
[(0, '0.048*"juego" + 0.026*"producto" + 0.021*"llegado" + 0.019*"caja" + 0.018*"día" + 0.016*"llegó" + 0.014*"amazon" + 0.013*"perfecto" + 0.010*"si" + 0.010*"ahora"'), (1, '0.045*"bien" + 0.025*"si" + 0.021*"mando" + 0.016*"calidad" + 0.016*"bonitos" + 0.014*"plata" + 0.013*"solo" + 0.012*"original" + 0.011*"buena" + 0.011*"pendientes"'), (2, '0.038*"calidad" + 0.025*"precio" + 0.024*"bien" + 0.024*"bonito" + 0.015*"foto" + 0

###  `[25 pts]` Punto 3: Visualización de LDA

In [59]:
display_list = []
for _model in models:
    lda_display = pyLDAvis.gensim.prepare(_model, corpus, dictionary, sort_topics=True)
    display_list.append(lda_display)

In [60]:
len(display_list)

4

### Visuzaliación 1
Si se evidencia que los grandes temas estan claramente separados y concuerdan con las dos categorias de productos => Video Juegos y Joyeria

In [61]:
pyLDAvis.display(display_list[0])

### Visualización 2

In [62]:
pyLDAvis.display(display_list[1])

### Visualización 3

In [63]:
pyLDAvis.display(display_list[2])

### Visualización 4

In [64]:
pyLDAvis.display(display_list[3])

Entre más temas quiero extraer se empiezan a cruzar las burbujas. Algo esperado.